In [8]:
import gurobipy as gp
from gurobipy import GRB

L, W, H = 12, 8, 8

# [cite_start]Box data: dimensions (length, width, height) for each box [cite: 156]
boxes_dims = [
    (6, 3, 2), (6, 3, 2),  # Boxes 1-2
    (6, 4, 3), (6, 4, 3), (6, 4, 3), (6, 4, 3), (6, 4, 3), # Boxes 3-7
    (8, 3, 2), (8, 3, 2), (8, 3, 2), # Boxes 8-10
    (4, 3, 2), (4, 3, 2), # Boxes 11-12
    (4, 4, 3), (4, 4, 3), (4, 4, 3)  # Boxes 13-15
]

# [cite_start]Number of available boxes (m) [cite: 73]
m = len(boxes_dims)

# [cite_start]Value of each box (v_i), set as its absolute volume [cite: 74]
values = [l*w*h/(L*W*H) for l,w,h in boxes_dims]

# --- 2. Parameters from the Formulation ---

# [cite_start]A sufficiently large number M for the big-M constraints [cite: 72]
M = L + W + H 

# [cite_start]Position of the container's origin [cite: 77]
Xo, Yo, Zo = 0, 0, 0

# --- 3. Model Setup ---

model = gp.Model("ImprovedContainerLoading")

# --- 4. Decision Variables ---

# [cite_start]p_i: 1 if box i is packed, 0 otherwise [cite: 79]
p = model.addVars(m, vtype=GRB.BINARY, name="p")

# (x_i, y_i, z_i)[cite_start]: front-left-bottom corner coordinates of box i [cite: 80]
x = model.addVars(m, vtype=GRB.CONTINUOUS, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, name="z")

# [cite_start]a_ij, b_ij, ...: relative position indicators [cite: 81]
pairs = [(i, j) for i in range(m) for j in range(m) if i < j]
a = model.addVars(pairs, vtype=GRB.BINARY, name="a")
b = model.addVars(pairs, vtype=GRB.BINARY, name="b")
c = model.addVars(pairs, vtype=GRB.BINARY, name="c")
d = model.addVars(pairs, vtype=GRB.BINARY, name="d")
e = model.addVars(pairs, vtype=GRB.BINARY, name="e")
f = model.addVars(pairs, vtype=GRB.BINARY, name="f")

# --- 5. Objective Function ---
# [cite_start]Maximize the total value (volume) of packed boxes [cite: 101]
model.setObjective(gp.quicksum(values[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# --- 6. Constraints ---

# A. Boundary and Placement Constraints
for i in range(m):
    li, wi, hi = boxes_dims[i]
                    # Box must be inside container IF PACKED (p_i=1)
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
            # Position variables are zero IF NOT PACKED (p_i=0)
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

        # B. Non-Overlapping Constraints (Improved Formulation)
for i, j in pairs:
        li, wi, hi = boxes_dims[i]
        lj, wj, hj = boxes_dims[j]
        
        model.addConstr(x[i] + li <= x[j] + M * (1 - a[i, j]), f"left_of_{i}_{j}")
        model.addConstr(x[j] + lj <= x[i] + M * (1 - b[i, j]), f"right_of_{i}_{j}")
        model.addConstr(y[i] + wi <= y[j] + M * (1 - c[i, j]), f"front_of_{i}_{j}")
        model.addConstr(y[j] + wj <= y[i] + M * (1 - d[i, j]), f"behind_{i}_{j}")
        model.addConstr(z[i] + hi <= z[j] + M * (1 - e[i, j]), f"below_{i}_{j}")
        model.addConstr(z[j] + hj <= z[i] + M * (1 - f[i, j]), f"above_{i}_{j}")

        # This constraint now only applies if BOTH boxes i and j are packed
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1, f"non_overlap_{i}_{j}")

        # C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes_dims):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                        # If both identical boxes are packed, enforce an arbitrary order
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]), f"sym_break_x_{i}_{j}")

        # --- 7. Solve the Model ---
# model.setParam('TimeLimit', 2)
model.setParam('MIPGap', 0.1) 
model.optimize()

        # --- 8. Display Results ---

print("\n" + "="*40)
print("OPTIMAL SOLUTION FOUND")
print("="*40)

utilization = (model.ObjVal) * 100

print(f"Container volume utilization: {utilization:.2f}%\n")

packed_boxes_indices = sorted([i for i in range(m) if p[i].X > 0.5])
unpacked_boxes_indices = sorted([i for i in range(m) if p[i].X < 0.5])

print(f"Packed {len(packed_boxes_indices)} of {m} boxes.")
print("-" * 30)
aux = 0
for i in packed_boxes_indices:
    aux += values[i]
    print(f"Box {i+1:<2}: Dims {str(boxes_dims[i]):<10} | Packed at (x={x[i].X:.1f}, y={y[i].X:.1f}, z={z[i].X:.1f})")

if unpacked_boxes_indices:
    print("\nUnpacked boxes (indices):")
    print([i + 1 for i in unpacked_boxes_indices])

print("\n" + "="*40)

print(aux)


Set parameter MIPGap to value 0.1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  0.1

Optimize a model with 843 rows, 690 columns and 2982 nonzeros
Model fingerprint: 0x7104672a
Variable types: 45 continuous, 645 integer (645 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-02, 9e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective 0.0468750
Presolve removed 104 rows and 59 columns
Presolve time: 0.01s
Presolved: 739 rows, 631 columns, 2649 nonzeros
Variable types: 45 continuous, 586 integer (586 binary)

Root relaxation: objective 1.000000e+00, 220 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | 

In [ ]:
import gurobipy as gp
from gurobipy import GRB

L, W, H = 12, 8, 8

# [cite_start]Box data: dimensions (length, width, height) for each box [cite: 156]
boxes_dims = [
    (6, 3, 2), (6, 3, 2),  # Boxes 1-2
    (6, 4, 3), (6, 4, 3), (6, 4, 3), (6, 4, 3), (6, 4, 3), # Boxes 3-7
    (8, 3, 2), (8, 3, 2), (8, 3, 2), # Boxes 8-10
    (4, 3, 2), (4, 3, 2), # Boxes 11-12
    (4, 4, 3), (4, 4, 3), (4, 4, 3)  # Boxes 13-15
]

# [cite_start]Number of available boxes (m) [cite: 73]
m = len(boxes_dims)

# [cite_start]Value of each box (v_i), set as its absolute volume [cite: 74]
values = [l*w*h/(L*W*H) for l,w,h in boxes_dims]

# --- 2. Parameters from the Formulation ---

# [cite_start]A sufficiently large number M for the big-M constraints [cite: 72]
M = L + W + H 

# [cite_start]Position of the container's origin [cite: 77]
Xo, Yo, Zo = 0, 0, 0

# --- 3. Model Setup ---

model = gp.Model("ImprovedContainerLoading")

# --- 4. Decision Variables ---

# [cite_start]p_i: 1 if box i is packed, 0 otherwise [cite: 79]
p = model.addVars(m, vtype=GRB.BINARY, name="p")

# (x_i, y_i, z_i)[cite_start]: front-left-bottom corner coordinates of box i [cite: 80]
x = model.addVars(m, vtype=GRB.CONTINUOUS, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, name="z")

# [cite_start]a_ij, b_ij, ...: relative position indicators [cite: 81]
pairs = [(i, j) for i in range(m) for j in range(m) if i < j]
a = model.addVars(pairs, vtype=GRB.BINARY, name="a")
b = model.addVars(pairs, vtype=GRB.BINARY, name="b")
c = model.addVars(pairs, vtype=GRB.BINARY, name="c")
d = model.addVars(pairs, vtype=GRB.BINARY, name="d")
e = model.addVars(pairs, vtype=GRB.BINARY, name="e")
f = model.addVars(pairs, vtype=GRB.BINARY, name="f")

# --- 5. Objective Function ---
# [cite_start]Maximize the total value (volume) of packed boxes [cite: 101]
model.setObjective(gp.quicksum(values[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

# --- 6. Constraints ---

# A. Boundary and Placement Constraints
for i in range(m):
    li, wi, hi = boxes_dims[i]
                    # Box must be inside container IF PACKED (p_i=1)
    model.addConstr(x[i] + li <= L + M * (1 - p[i]), f"bound_x_{i}")
    model.addConstr(y[i] + wi <= W + M * (1 - p[i]), f"bound_y_{i}")
    model.addConstr(z[i] + hi <= H + M * (1 - p[i]), f"bound_z_{i}")
            
            # Position variables are zero IF NOT PACKED (p_i=0)
    model.addConstr(x[i] <= M * p[i], f"pos_x_zero_{i}")
    model.addConstr(y[i] <= M * p[i], f"pos_y_zero_{i}")
    model.addConstr(z[i] <= M * p[i], f"pos_z_zero_{i}")

        # B. Non-Overlapping Constraints (Improved Formulation)
for i, j in pairs:
        li, wi, hi = boxes_dims[i]
        lj, wj, hj = boxes_dims[j]
        
        model.addConstr(x[i] + li <= x[j] + M * (1 - a[i, j]), f"left_of_{i}_{j}")
        model.addConstr(x[j] + lj <= x[i] + M * (1 - b[i, j]), f"right_of_{i}_{j}")
        model.addConstr(y[i] + wi <= y[j] + M * (1 - c[i, j]), f"front_of_{i}_{j}")
        model.addConstr(y[j] + wj <= y[i] + M * (1 - d[i, j]), f"behind_{i}_{j}")
        model.addConstr(z[i] + hi <= z[j] + M * (1 - e[i, j]), f"below_{i}_{j}")
        model.addConstr(z[j] + hj <= z[i] + M * (1 - f[i, j]), f"above_{i}_{j}")

        # This constraint now only applies if BOTH boxes i and j are packed
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1, f"non_overlap_{i}_{j}")

        # C. Symmetry-Breaking Constraints
box_groups = {}
for i, dims in enumerate(boxes_dims):
    dim_key = tuple(dims)
    if dim_key not in box_groups:
        box_groups[dim_key] = []
    box_groups[dim_key].append(i)

for dims, indices in box_groups.items():
    if len(indices) > 1:
        for i_idx in range(len(indices) - 1):
            for j_idx in range(i_idx + 1, len(indices)):
                i = indices[i_idx]
                j = indices[j_idx]
                        # If both identical boxes are packed, enforce an arbitrary order
                model.addConstr(x[i] <= x[j] + M * (2 - p[i] - p[j]), f"sym_break_x_{i}_{j}")

        # --- 7. Solve the Model ---
# model.setParam('TimeLimit', 2)
model.setParam('MIPGap', 0.1) 
model.optimize()

        # --- 8. Display Results ---

print("\n" + "="*40)
print("OPTIMAL SOLUTION FOUND")
print("="*40)

utilization = (model.ObjVal) * 100

print(f"Container volume utilization: {utilization:.2f}%\n")

packed_boxes_indices = sorted([i for i in range(m) if p[i].X > 0.5])
unpacked_boxes_indices = sorted([i for i in range(m) if p[i].X < 0.5])

print(f"Packed {len(packed_boxes_indices)} of {m} boxes.")
print("-" * 30)
aux = 0
for i in packed_boxes_indices:
    aux += values[i]
    print(f"Box {i+1:<2}: Dims {str(boxes_dims[i]):<10} | Packed at (x={x[i].X:.1f}, y={y[i].X:.1f}, z={z[i].X:.1f})")

if unpacked_boxes_indices:
    print("\nUnpacked boxes (indices):")
    print([i + 1 for i in unpacked_boxes_indices])

print("\n" + "="*40)

print(aux)


Set parameter MIPGap to value 0.1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  0.1

Optimize a model with 843 rows, 690 columns and 2982 nonzeros
Model fingerprint: 0x7104672a
Variable types: 45 continuous, 645 integer (645 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-02, 9e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective 0.0468750
Presolve removed 104 rows and 59 columns
Presolve time: 0.01s
Presolved: 739 rows, 631 columns, 2649 nonzeros
Variable types: 45 continuous, 586 integer (586 binary)

Root relaxation: objective 1.000000e+00, 220 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | 

In [5]:
tipo_dict = {}
tipo_counter = 1
boxes_dims = [
            (6, 3, 2), (6, 3, 2),  # Boxes 1-2
            (6, 4, 3), (6, 4, 3), (6, 4, 3), (6, 4, 3), (6, 4, 3), # Boxes 3-7
            (8, 3, 2), (8, 3, 2), (8, 3, 2), # Boxes 8-10
            (4, 3, 2), (4, 3, 2), # Boxes 11-12
            (4, 4, 3), (4, 4, 3), (4, 4, 3)  # Boxes 13-15
        ]

for box in boxes_dims:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1

# Imprimir saída no formato correto
print(f"{L} {W} {H}")  # primeira linha: dimensões do contêiner

for i in range(m):
    li, wi, hi = boxes_dims[i]
    x_i = x[i].X if p[i].X > 0.5 else 0
    y_i = y[i].X if p[i].X > 0.5 else 0
    z_i = z[i].X if p[i].X > 0.5 else 0
    tipo = tipo_dict[boxes_dims[i]]  # tipo real baseado nas dimensões
    cliente = 1
    print(f"{x_i} {y_i} {z_i} {li} {wi} {hi} {tipo} {cliente}")

with open('dados.txt','w') as f:
    f.write(f"{L} {W} {H}\n")  # primeira linha: dimensões do contêiner
    for i in range(m):
        if p[i].X > 0.5:
            li, wi, hi = boxes_dims[i]
            x_i, y_i, z_i = x[i].X, y[i].X, z[i].X
            tipo = tipo_dict[boxes_dims[i]]  # tipo baseado nas dimensões
            cliente = 1
            f.write(f"{x_i} {y_i} {z_i} {li} {wi} {hi} {tipo} {cliente}\n")

12 8 8
0 0 0 6 3 2 1 1
6.0 0.0 6.0 6 3 2 1 1
0.0 4.0 0.0 6 4 3 2 1
0.0 0.0 5.0 6 4 3 2 1
0.0 4.0 3.0 6 4 3 2 1
6.0 4.0 3.0 6 4 3 2 1
6.0 4.0 0.0 6 4 3 2 1
0.0 0.0 3.0 8 3 2 3 1
4.0 4.0 6.0 8 3 2 3 1
0 0 0 8 3 2 3 1
0.0 5.0 6.0 4 3 2 4 1
8.0 0.0 0.0 4 3 2 4 1
0.0 0.0 0.0 4 4 3 5 1
4.0 0.0 0.0 4 4 3 5 1
8.0 0.0 2.0 4 4 3 5 1
